In [ ]:
import numpy as np
from numpy import linalg as LA
from sklearn.neighbors import kneighbors_graph
import torch


In [ ]:
def knn_find(points, k: int):
    """find indexes of k nearest points for every point in "points" array

        Parameters
        ----------
        points   : np.array 
                   set of 3d points [N*3]
        k        : int
                   number of nearest neighbours

        Returns
        -------
        tensor [N*k] - for every point indexes of k nearest neighbours
        """

    graph = kneighbors_graph(
        points, k, mode='connectivity', include_self=False)
    array_neighbours = graph.toarray()

    # return torch.from_numpy(array_neighbours)
    return array_neighbours


def kernel(weights, point):
    """calculates H(W, X)
    """

    new_point = np.insert(point, 0, 1., axis=0)
    answer = np.dot(weights, new_point)/LA.norm(weights[1:])
#     print(answer)
    return answer


def final_kernel(points, i, weights, channel, k, id_neighbours):
    """calculates 1/(1 + exp(H(W, X)))
    """

    pc_first = 1/k*sum([kernel(weights[channel, :], points[id_neighbour] - points[i])
                        for id_neighbour in id_neighbours])
    pc_final = 1/(1 + np.power(2.73, pc_first))
    return pc_final


def convolution(points, k, weights, channels):
    """creates features for every point
    Parameters
        ----------
        points   : torch.tensor
                   set of 3d points [N*3]
        k        : int
                   number of nearest neighbours
        weights  : torch.tensor
                   set of weights [channels*4]
        channels : int
                   number of channels

        Returns
        -------
        tensor [N*channels] - for every point "channels" features
    
    """
    points_array = points.numpy()  # torch tensor to np.array
    weights = weights.numpy()
    number_points = points.shape[0]
    knn_array = knn_find(points_array, k)
    array_features = np.zeros((number_points, channels))
    for i in range(number_points):
        id_neighbours = np.nonzero(knn_array[i])[0]
        array_features[i] = np.asarray([final_kernel(
            points_array, i, weights, channel, k, id_neighbours) for channel in np.arange(0, channels, 1)])
    return array_features

In [ ]:
points = torch.rand(1000,3)
weights = torch.rand(4,4)
a = convolution(points, 3, weights, 4)